# VacationPy
### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df=pd.read_csv('../WeatherPy/cities_df.csv')
vacation_df

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind
0,0,georgetown,MY,5.4112,100.3354,86.90,78,40,1.01
1,1,amapa,BR,1.0000,-52.0000,70.21,97,64,2.21
2,2,vaini,TO,-21.2000,-175.2000,77.16,78,75,12.66
3,3,ushuaia,AR,-54.8000,-68.3000,33.46,93,75,4.61
4,4,hobart,AU,-42.8794,147.3294,61.27,56,75,1.99
...,...,...,...,...,...,...,...,...,...
602,602,kansanshi,ZM,-12.0951,26.4273,46.60,67,100,7.67
603,603,tahoua,NE,14.8888,5.2692,73.65,82,100,20.67
604,604,tymovskoye,RU,50.8456,142.6550,87.03,52,99,4.97
605,605,gulu,UG,2.7746,32.2990,65.89,69,99,3.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#url=f"krrps://maps.gooleapis.com/maps.api

# Store 'Lat' and 'Lng' into  locations and humidity.
locations = vacation_df[["Latitude", "Longitude"]].astype(float)
humidity = vacation_df['Humidity'].astype(float)
maxhumidity = humidity.max()

# Make the base map.
fig = gmaps.figure()
#Add the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius = 2)

fig.add_layer(heat_layer)
#print the figure.
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
vacation_thoughts_df = vacation_df.loc[(vacation_df['Temperature']>65)&(vacation_df['Temperature']<80)&(vacation_df['Wind']<10)&(vacation_df['Humidity']<35)&(vacation_df['Cloudiness']<20),:]
vacation_thoughts_df.reset_index(drop=True)


,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind
0,197,port elizabeth,ZA,-33.9180,25.5701,71.10,33,0,6.91
1,263,gatton,AU,-27.5500,152.2833,75.40,27,0,2.53
2,474,malazgirt,TR,39.1427,42.5401,69.57,30,0,2.57
3,559,shiraz,IR,29.6036,52.5388,76.44,22,0,0.00


## Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df=pd.DataFrame(vacation_thoughts_df, columns=['City','Country','Latitude','Longitude','Humidity','Cloudiness','Wind'])

hotel_df['Hotel Name']=""
hotel_df

,City,Country,Latitude,Longitude,Humidity,Cloudiness,Wind,Hotel Name
197,port elizabeth,ZA,-33.9180,25.5701,33,0,6.91,
263,gatton,AU,-27.5500,152.2833,27,0,2.53,
474,malazgirt,TR,39.1427,42.5401,30,0,2.57,
559,shiraz,IR,29.6036,52.5388,22,0,0.00,


In [14]:
hotels = []

for i in range(len(hotel_df)):
    lat=hotel_df.iloc[i]['Latitude']
    lng=hotel_df.iloc[i]['Longitude']
    
    params = {
        "location":f'{lat},{lng}',
        "radius":5000,
        "types":"hotel",
        "key":g_key
    }
    
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response =  requests.get(url,params=params).json()
    
    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("")
hotel_df["Hotel Name"]=hotels
hotel_df

,City,Country,Latitude,Longitude,Humidity,Cloudiness,Wind,Hotel Name
197,port elizabeth,ZA,-33.9180,25.5701,33,0,6.91,Gqeberha
263,gatton,AU,-27.5500,152.2833,27,0,2.53,Gatton
474,malazgirt,TR,39.1427,42.5401,30,0,2.57,Malazgirt
559,shiraz,IR,29.6036,52.5388,22,0,0.00,Shiraz


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
